In [57]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster

from utils import *
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin', 'C:/Users/j

In [58]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [59]:
stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"
elif response_locked:
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"

In [60]:
stimulus_experiment_conditions = {
    "Stimulus_i25s25": {
        "BIDS_events": "Stimulus/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_i25s75": {
        "BIDS_events": "Stimulus/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_i75s25": {
        "BIDS_events": "Stimulus/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_i75s75": {
        "BIDS_events": "Stimulus/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_i25r25": {
        "BIDS_events": "Stimulus/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_i25r75": {
        "BIDS_events": "Stimulus/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus_i75r25": {
        "BIDS_events": "Stimulus/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_i75r75": {
        "BIDS_events": "Stimulus/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus_c25s25": {
        "BIDS_events": "Stimulus/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_c25s75": {
        "BIDS_events": "Stimulus/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_c75s25": {
        "BIDS_events": "Stimulus/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_c75s75": {
        "BIDS_events": "Stimulus/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_c25r25": {
        "BIDS_events": "Stimulus/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_c25r75": {
        "BIDS_events": "Stimulus/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus_c75r25": {
        "BIDS_events": "Stimulus/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_c75r75": {
        "BIDS_events": "Stimulus/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

# congruency_conditions = {
#     "Stimulus_c": {
#         "BIDS_events": ["Stimulus/c25/s25", "Stimulus/c25/s75", "Stimulus/c75/s25", "Stimulus/c75/s75", "Stimulus/c25/r25", "Stimulus/c25/r75", "Stimulus/c75/r25", "Stimulus/c75/r75"],
#         "congruency": "c"
#     }
# }

stimulus_conditions = {
    "Stimulus_bigSsmallHtaskG": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus_bigSsmallHtaskL": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus_bigSsmallStaskG": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus_bigSsmallStaskL": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Stimulus_bigHsmallHtaskG": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus_bigHsmallHtaskL": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus_bigHsmallStaskG": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus_bigHsmallStaskL": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

response_experiment_conditions = {
    "Response_i25s25": {
        "BIDS_events": "Response/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response_i25s75": {
        "BIDS_events": "Response/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response_i75s25": {
        "BIDS_events": "Response/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response_i75s75": {
        "BIDS_events": "Response/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response_i25r25": {
        "BIDS_events": "Response/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response_i25r75": {
        "BIDS_events": "Response/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response_i75r25": {
        "BIDS_events": "Response/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response_i75r75": {
        "BIDS_events": "Response/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response_c25s25": {
        "BIDS_events": "Response/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response_c25s75": {
        "BIDS_events": "Response/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response_c75s25": {
        "BIDS_events": "Response/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response_c75s75": {
        "BIDS_events": "Response/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response_c25r25": {
        "BIDS_events": "Response/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response_c25r75": {
        "BIDS_events": "Response/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response_c75r25": {
        "BIDS_events": "Response/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response_c75r75": {
        "BIDS_events": "Response/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

response_conditions = {
    "Response_bigSsmallHtaskG": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Response_bigSsmallHtaskL": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Response_bigSsmallStaskG": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Response_bigSsmallStaskL": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Response_bigHsmallHtaskG": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Response_bigHsmallHtaskL": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Response_bigHsmallStaskG": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Response_bigHsmallStaskL": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

In [61]:
task='GlobalLocal'

if stimulus_locked:
    conditions = stimulus_conditions
elif response_locked:
    conditions = response_conditions #set this to whichever you're running

condition_names = list(conditions.keys()) # get the condition names as a list

# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Loading data for subject: D0057
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
  Loading condition: Stimulus_bigSsmallHtaskG with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetterh/Taskg', 'bigLetter': 's', 'smallLetter': 'h', 'task': 'g'}
  Loading condition: Stimulus_bigSsmallHtaskL with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetterh/Taskl', 'bigLetter': 's', 'smallLetter': 'h', 'task': 'l'}
  Loading condition: Stimulus_bigSsmallStaskG with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetters/Taskg', 'bigLetter': 's', 'smallLetter': 's', 'task': 'g'}
  Loading condition: Stimulus_bi

In [62]:
# import mne
# import numpy as np
# from rsatoolbox.data import TemporalDataset

# def create_temporal_dataset(data_dict):
#     """
#     Create a TemporalDataset from a dictionary of Epochs data.
    
#     Parameters:
#     data_dict (dict): Dictionary with the structure subject -> condition -> HG_ev1_rescaled.
    
#     Returns:
#     TemporalDataset: The rsatoolbox TemporalDataset object created from the dictionary data.
#     """
#     # Initialize lists to hold the data and labels
#     data = []
#     obs_descriptors = {'subject': [], 'condition': []}
#     channel_descriptors = {}
#     time_descriptors = {}

#     # Extract the data
#     for subject, conditions in data_dict.items():
#         for condition, epochs_dict in conditions.items():
#             epochs = epochs_dict['HG_ev1_rescaled']
#             # Get the data from the Epochs object
#             epochs_data = epochs.get_data()  # shape: (n_epochs, n_channels, n_times)
            
#             # Append data and descriptors
#             data.append(epochs_data)
#             obs_descriptors['subject'].extend([subject] * epochs_data.shape[0])
#             obs_descriptors['condition'].extend([condition] * epochs_data.shape[0])

#     # Convert lists to numpy arrays
#     data = np.concatenate(data, axis=0)  # shape: (total_epochs, n_channels, n_times)

#     # Assuming the first Epochs object has the relevant info for channels and times
#     sample_epochs = next(iter(next(iter(data_dict.values())).values()))['HG_ev1_rescaled']
#     channel_descriptors['name'] = sample_epochs.ch_names
#     time_descriptors['times'] = sample_epochs.times

#     # Create TemporalDataset
#     dataset = TemporalDataset(
#         measurements=data,
#         obs_descriptors=obs_descriptors,
#         channel_descriptors=channel_descriptors,
#         time_descriptors=time_descriptors
#     )

#     return dataset

# # Example usage:
# dataset = create_temporal_dataset(subjects_mne_objects)
# print(dataset)


get significant channels

In [63]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


In [64]:
# Assuming you have a list of subjects
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
root_dir = rf"C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs"


In [65]:
# Define your ROIs dictionary and other parameters
rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
}

rois = list(rois_dict.keys())
# Assuming you have subjects_electrodestoROIs_dict and sig_chans_per_subject dictionaries
electrodes_per_subject_roi, sig_electrodes_per_subject_roi,  = make_sig_electrodes_per_subject_and_roi_dict(
    rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject
)


For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF16', 'LASF10', 'LASF14', 'LASF15', 'LASF16', 'LM

get electrode counts for each roi

In [66]:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])


Total number of lpfc electrodes across all subjects: 105
Total number of significant lpfc electrodes across all subjects: 46
Total number of v1 electrodes across all subjects: 17
Total number of significant v1 electrodes across all subjects: 12
Total number of occ electrodes across all subjects: 57
Total number of significant occ electrodes across all subjects: 41


check if any subjects have a weird sampling rate

In [67]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)


Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.


create a dat dict like in the Temporal example from rsatoolbox 7/19  
#### Data Dictionary (dat) Structure
- **roi1**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roi2**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roiX**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample


        




In [68]:
sig_electrodes = sig_electrodes_per_subject_roi['lpfc'].get('D0059', [])
sig_electrodes

['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']

In [86]:
dat['occ']['condition_names']

{'Stimulus_bigSsmallHtaskG': 0,
 'Stimulus_bigSsmallHtaskL': 1,
 'Stimulus_bigSsmallStaskG': 2,
 'Stimulus_bigSsmallStaskL': 3,
 'Stimulus_bigHsmallHtaskG': 4,
 'Stimulus_bigHsmallHtaskL': 5,
 'Stimulus_bigHsmallStaskG': 6,
 'Stimulus_bigHsmallStaskL': 7}

In [69]:
import numpy as np

def prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi):
    # returns dat
    dat = {}
    overall_electrode_mapping = []
    electrode_mapping_per_roi = {roi: [] for roi in rois}  # Reinitialize for each processing run
    print('subjects: ', subjects)
    for roi in rois:
        dat[roi] = {}  # make a dict for each roi
        dat[roi]['channel_names'] = []  # initialize a list to hold channel names
        dat[roi]['channel_rois'] = [] # initialize a list to hold what roi each channel is a part of
        dat[roi]['condition_names'] = {}  # initialize dict where keys are condition names and values are integer indices
        dat[roi]['cond_idx'] = np.array([], dtype=int)  # initialize an empty 1D array for condition indices for each trial
        dat[roi]['sub_idx'] = np.array([], dtype=int) # initialize an empty 1D array for subject for each trial
        dat[roi]['times'] = np.array([])  # initialize an empty 1D array for time points

        # Determine all unique channels across subjects for this ROI, maintaining order
        all_channels = []
        for sub in subjects:
            sig_electrodes = sig_electrodes_per_subject_roi[roi].get(sub, [])
            sub_channel_names = [sub + '-' + sig_electrode for sig_electrode in sig_electrodes]
            for chan in sub_channel_names:
                if chan not in all_channels:
                    all_channels.append(chan)
                    dat[roi]['channel_rois'].append(roi) # append roi for each channel
        dat[roi]['channel_names'] = all_channels
        num_channels = len(all_channels)
        print('num channels: ', num_channels)

        # Initialize the data array with the number of trials and total channels
        dat[roi]['data'] = np.empty((0, num_channels, 0))  # initialize an empty 3D array for trials x channels x time points

        for sub in subjects:
            total_trials = 0  # Initialize counter for total trials across all conditions
            sig_electrodes = sig_electrodes_per_subject_roi[roi].get(sub, [])
            sub_channel_names = [sub + '-' + sig_electrode for sig_electrode in sig_electrodes]
            if not sig_electrodes:
                continue

            cond_idx = 0  # the example uses indexing from 1, but let's start from 0 because python
            for condition_name in condition_names:
                print(f'Processing {sub} for {condition_name} in {roi}')
                epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_rescaled'].copy().pick(sig_electrodes)
                dat[roi]['condition_names'][condition_name] = cond_idx

                epochs_data = epochs.get_data(copy=True)
                num_trials, num_sub_channels, num_timepoints = epochs_data.shape

                print(f'Number of trials for {sub} in {condition_name}: {num_trials}')
                total_trials += num_trials

                # Initialize data array time dimension if it is empty
                if dat[roi]['data'].shape[2] == 0:
                    dat[roi]['data'] = np.empty((0, num_channels, num_timepoints))
                    dat[roi]['times'] = epochs.times

                # Create an array filled with NaNs for the current subject's data
                sub_data = np.full((num_trials, num_channels, num_timepoints), np.nan)

                # Find the indices for the subject's channels in the total list of channels
                channel_indices = [all_channels.index(chan) for chan in sub_channel_names]
                print('sub: ', sub)
                print("channel indices: ", channel_indices)
                
                # Place the subject's data in the correct indices
                sub_data[:, channel_indices, :] = epochs_data

                # Concatenate the new data along the first axis (trials)
                dat[roi]['data'] = np.concatenate((dat[roi]['data'], sub_data), axis=0)

                # Extend the cond_idx array
                dat[roi]['cond_idx'] = np.concatenate((dat[roi]['cond_idx'], np.full(num_trials, cond_idx)))

                # extend the sub_idx array by this number of trials with this subject
                dat[roi]['sub_idx'] = np.concatenate((dat[roi]['sub_idx'], np.full(num_trials, sub)))
                print('last entry in cond_idx is: ', dat[roi]['cond_idx'][-1])
                print('cond_idx is: ', cond_idx)
                cond_idx += 1  # increment cond_idx

            print(f'Total number of trials for {sub} across all conditions: {total_trials}')

    return dat

# Example call to the function (you need to replace the arguments with actual data)
dat = prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)


subjects:  ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
num channels:  46
Processing D0057 for Stimulus_bigSsmallHtaskG in lpfc
Number of trials for D0057 in Stimulus_bigSsmallHtaskG: 59
sub:  D0057
channel indices:  [0, 1]
last entry in cond_idx is:  0
cond_idx is:  0
Processing D0057 for Stimulus_bigSsmallHtaskL in lpfc
Number of trials for D0057 in Stimulus_bigSsmallHtaskL: 54
sub:  D0057
channel indices:  [0, 1]
last entry in cond_idx is:  1
cond_idx is:  1
Processing D0057 for Stimulus_bigSsmallStaskG in lpfc
Number of trials for D0057 in Stimulus_bigSsmallStaskG: 53
sub:  D0057
channel indices:  [0, 1]
last entry in cond_idx is:  2
cond_idx is:  2
Processing D0057 for Stimulus_bigSsmallStaskL in lpfc
Number of trials for D0057 in Stimulus_bigSsmallStaskL: 34
sub:  D0057
channel indices:  [0, 1]
last entry in cond_idx is:  3
cond_idx is:  3
Processing D0057 for Stimulus_bigHsmallHtaskG in lpfc
Number of trials for D00

plot condition averages for two channels in occ

In [70]:
# Extract the relevant data from the dictionary
occ_chan_names = dat['occ']['channel_names']
occ_data = dat['occ']['data']
occ_cond_idx = dat['occ']['cond_idx']
occ_times = dat['occ']['times']
occ_cond_names = [x for x in dat['occ']['condition_names'].keys()]

# Create the subplots
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax = ax.flatten()

# Plot the data
for jj, chan in enumerate(occ_chan_names[:2]):
    for ii, cond_ii in enumerate(np.unique(occ_cond_idx)):
        mn = np.nanmean(occ_data[cond_ii == occ_cond_idx, jj, :], axis=0).squeeze()
        ax[jj].plot(occ_times, mn, label=occ_cond_names[ii])
        ax[jj].set_title(chan)

# Add the legend
ax[0].legend()
# plt.show()


plot big s vs big h averaged over channels and trials in occ

In [71]:
import matplotlib.pyplot as plt
import numpy as np

# Extract the relevant data from the dictionary
occ_chan_names = dat['occ']['channel_names']
occ_data = dat['occ']['data']
occ_cond_idx = dat['occ']['cond_idx']
occ_times = dat['occ']['times']
occ_cond_names = [x for x in dat['occ']['condition_names'].keys()]

# Group condition indices by whether they contain 'bigH' or 'bigS'
bigH_conditions = [name for name in occ_cond_names if 'bigH' in name]
bigS_conditions = [name for name in occ_cond_names if 'bigS' in name]

bigH_indices = [dat['occ']['condition_names'][name] for name in bigH_conditions]
bigS_indices = [dat['occ']['condition_names'][name] for name in bigS_conditions]

# Calculate the mean and standard error over conditions for each group
bigH_data = np.concatenate([occ_data[occ_cond_idx == idx, :, :] for idx in bigH_indices], axis=0)
bigS_data = np.concatenate([occ_data[occ_cond_idx == idx, :, :] for idx in bigS_indices], axis=0)

# Calculate the mean over trials for each group and channel
mean_bigH = np.nanmean(bigH_data, axis=0)
mean_bigS = np.nanmean(bigS_data, axis=0)

# Calculate standard error over the trials for each channel and time point
sem_bigH = np.nanstd(bigH_data, axis=0, ddof=1) / np.sqrt(bigH_data.shape[0])
sem_bigS = np.nanstd(bigS_data, axis=0, ddof=1) / np.sqrt(bigS_data.shape[0])

# Average the mean and SEM across channels for plotting
mean_bigH = np.nanmean(mean_bigH, axis=0)
mean_bigS = np.nanmean(mean_bigS, axis=0)
sem_bigH = np.nanmean(sem_bigH, axis=0)
sem_bigS = np.nanmean(sem_bigS, axis=0)

# Plot the results with standard error shading
plt.figure(figsize=(12, 6))
plt.plot(occ_times, mean_bigH, label='bigH', color='green', linestyle='-')
plt.plot(occ_times, mean_bigS, label='bigS', color='green', linestyle='--')
plt.fill_between(occ_times, mean_bigH - sem_bigH, mean_bigH + sem_bigH, color='green', alpha=0.3)
plt.fill_between(occ_times, mean_bigS - sem_bigS, mean_bigS + sem_bigS, color='green', alpha=0.3)
plt.xlabel('Time from Stim Onset (s)')
plt.ylabel('Z-score')
plt.title('Occ Average over bigH vs bigS conditions')
plt.legend()
plt.ylim([-0.2, 0.5])
plt.show()


turn the power trace plotting into a function 7/21

In [76]:
# Function to compare conditions and save the plots
def compare_conditions_and_save(dat, rois, save_dir, colors, linestyles, conditions_to_be_plotted):
    for roi in rois:
        # Extract the relevant data for the current ROI
        roi_chan_names = dat[roi]['channel_names']
        roi_data = dat[roi]['data']
        roi_cond_idx = dat[roi]['cond_idx']
        roi_times = dat[roi]['times']
        roi_cond_names = [x for x in dat[roi]['condition_names'].keys()]

        for label, condition_dict in conditions_to_be_plotted.items():
            plt.figure(figsize=(12,6))
            for cond_name, cond_values in condition_dict.items():
                if isinstance(cond_values, str):
                    cond_values = [cond_values]
                    
                # Group condition indices by whether they contain any of the specified strings in the list
                conditions = [name for name in roi_cond_names if any(c in name for c in cond_values)]
                condition_indices = [dat[roi]['condition_names'][name] for name in conditions]

                # Calculate the mean and standard error over conditions for each group
                condition_data = np.concatenate([roi_data[roi_cond_idx == idx, :, :] for idx in condition_indices], axis=0)

                # Calculate the mean over trials for each group and channel
                mean_condition = np.nanmean(condition_data, axis=0)

                # Calculate standard error over the trials for each channel and time point
                sem_condition = np.nanstd(condition_data, axis=0, ddof=1) / np.sqrt(condition_data.shape[0])

                # Average the mean and SEM across channels for plotting
                mean_condition = np.nanmean(mean_condition, axis=0)
                sem_condition = np.nanmean(sem_condition, axis=0)

                # Plot the results with standard error shading
                plt.plot(roi_times, mean_condition, label=cond_name, color=colors[label], linestyle=linestyles[cond_name])
                plt.fill_between(roi_times, mean_condition - sem_condition, mean_condition + sem_condition, color=colors[label], alpha=0.3)

            plt.xlabel('Time from Stim Onset (s)')
            plt.ylabel('Z-score')
            plt.title(f'{roi} Average over conditions')
            plt.legend()
            plt.ylim([-1, 1])

            # Save the figure
            filename = f'{roi}_{label}_rsatoolbox_power_plot.png'
            plt.savefig(os.path.join(save_dir, filename))
            plt.close()

In [77]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
save_dir = os.path.join(LAB_root, 'BIDS-1.1_GlobalLocal', 'BIDS', 'derivatives', 'RSA', 'figs')
print(save_dir)
rois = ['lpfc', 'v1', 'occ']

# Define colors for the model names
colors = {
    'congruency': 'red',
    'switchType': 'blue',
    'congruencyProportion': 'pink',
    'switchProportion': 'skyblue',
    'congruency_congruencyProportion': 'hotpink',
    'switchType_switchProportion': 'gray',
    'bigLetter': 'green',
    'smallLetter': 'orange',
    'task': 'gray',
}

# Define linestyles for the model names
linestyles = {
    'big letter S': '-',
    'big letter H': '--',
    'small letter S': '-.',
    'small letter H': ':',
    'task G': '-',
    'task L': '--',
    'congruent': '-',
    'incongruent': '--',
}

if conditions == stimulus_conditions:
    conditions_to_be_plotted = {
        'bigLetter': {'big letter S': 'bigS', 'big letter H': 'bigH'},
        'smallLetter': {'small letter S': 'smallS', 'small letter H': 'smallH'},
        'task': {'task G': 'taskG', 'task L': 'taskL'},
    }
elif conditions == stimulus_experiment_conditions:

    conditions_to_be_plotted = {
        'congruency': {'congruent': ['c25', 'c75'], 'incongruent': ['i25', 'i75']},
        'switchType': {'switch': ['s25', 's75'], 'repeat': ['r25', 'r75']}
    }

compare_conditions_and_save(dat, rois, save_dir, colors, linestyles, conditions_to_be_plotted)

C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\RSA\figs


set up TemporalDataset objects

In [78]:
temporal_datasets = {}

for roi in rois:
    measurements = dat[roi]['data']
    des = {'roi': roi}
    obs_des = {'cond_idx': dat[roi]['cond_idx'], 'sub_idx': dat[roi]['sub_idx']}
    chn_des = {'channel_names': dat[roi]['channel_names'], 'channel_rois': dat[roi]['channel_rois']}
    tim_des = {'time': dat[roi]['times']}

    temporal_datasets[roi] = rsatoolbox.data.TemporalDataset(measurements, 
                                             descriptors = des, 
                                             obs_descriptors = obs_des, 
                                             channel_descriptors = chn_des,
                                             time_descriptors = tim_des)


bin the times (just for occ for now, go back and make this loop over rois) 7/21

In [81]:
import numpy as np

# Assuming temporal_datasets is already created and contains 'occ' key
time_points = temporal_datasets['occ'].time_descriptors['time']

# Define the bins: 100 ms each
bin_size = 0.1  # 100 ms
bins = []

# Find the start and end times from the time_points array
start_time = np.min(time_points)
end_time = np.max(time_points)

current_time = start_time
while current_time < end_time:
    next_time = current_time + bin_size
    if next_time > end_time:
        next_time = end_time
    bins.append(time_points[(time_points >= current_time) & (time_points < next_time)])
    current_time = next_time

# Ensure that the last bin includes the endpoint
if time_points[-1] not in bins[-1]:
    bins[-1] = np.append(bins[-1], time_points[-1])


print("First bin:", bins[0])
print("Last bin:", bins[-1])

# # Perform the binning
# occ_data_binned = temporal_datasets['occ'].bin_time('time', bins=bins)

# print('shape before binning: ', temporal_datasets['occ'].measurements.shape)
# print('\nshape after binning')
# print(occ_data_binned.measurements.shape)
# print(occ_data_binned.time_descriptors['time'][0])

# # To verify:
# print('Number of bins:', len(occ_data_binned.time_descriptors['time']))
# print('Binned time points:', occ_data_binned.time_descriptors['time'])


First bin: [-1.         -0.99609375 -0.9921875  -0.98828125 -0.984375   -0.98046875
 -0.9765625  -0.97265625 -0.96875    -0.96484375 -0.9609375  -0.95703125
 -0.953125   -0.94921875 -0.9453125  -0.94140625 -0.9375     -0.93359375
 -0.9296875  -0.92578125 -0.921875   -0.91796875 -0.9140625  -0.91015625
 -0.90625    -0.90234375]
Last bin: [1.40234375 1.40625    1.41015625 1.4140625  1.41796875 1.421875
 1.42578125 1.4296875  1.43359375 1.4375     1.44140625 1.4453125
 1.44921875 1.453125   1.45703125 1.4609375  1.46484375 1.46875
 1.47265625 1.4765625  1.48046875 1.484375   1.48828125 1.4921875
 1.49609375 1.5       ]
shape before binning:  (3011, 41, 641)

shape after binning
(3011, 41, 25)
-0.951171875
Number of bins: 25
Binned time points: [-0.95117188 -0.84960938 -0.75       -0.65039062 -0.55078125 -0.45117188
 -0.34960938 -0.25       -0.15039062 -0.05078125  0.04882812  0.15039062
  0.25        0.34960938  0.44921875  0.54882812  0.65039062  0.75
  0.84960938  0.94921875  1.04882812

try make rdm (just do occ for testing)

In [82]:
rdms_occ_data = calc_rdm_movie(temporal_datasets['occ'], method='correlation', descriptor='cond_idx', bins=bins, unbalanced=True)
rdms_occ_data

rsatoolbox.rdm.RDMs(
dissimilarity_measure = 
correlation
dissimilarities = 
[[0.15609221 0.16844275 0.12326141 0.17540432 0.215364   0.15507777
  0.17987067 0.19443647 0.1898831  0.3117649  0.2529446  0.15159027
  0.22383369 0.09066612 0.217346   0.23553883 0.0964933  0.14560216
  0.20195237 0.21389387 0.09512741 0.11451478 0.15702928 0.24172767
  0.24824444 0.16975176 0.34517226 0.21608817]
 [0.10301024 0.13498092 0.09675612 0.18517931 0.22350621 0.13570095
  0.10862354 0.16342879 0.14188816 0.16470774 0.23326123 0.1525438
  0.14036048 0.14405265 0.18968905 0.19316756 0.08572742 0.11845864
  0.23878332 0.17003539 0.19756337 0.13055328 0.24500841 0.24856448
  0.15002387 0.19470083 0.25171176 0.16208127]
 [0.13319894 0.170232   0.15619747 0.17718386 0.15413497 0.08167534
  0.09219365 0.18255965 0.19358724 0.13477398 0.20448481 0.16275176
  0.09351857 0.09956711 0.1480533  0.19078966 0.12967979 0.11930594
  0.12817677 0.12745241 0.13314975 0.09835553 0.11938424 0.24566709
  0.13639925 0

In [85]:
plt.figure(figsize=(10,15))

# add formated time as rdm_descriptor
rdms_occ_data.rdm_descriptors['time_formatted'] = ['%0.0f ms' % (np.round(x*1000,2)) for x in rdms_occ_data.rdm_descriptors['time']]

rsatoolbox.vis.show_rdm(rdms_occ_data, 
                   pattern_descriptor='cond_idx',
                   rdm_descriptor='time_formatted')
plt.show()

KeyboardInterrupt: 

In [ ]:
def debugger_test():
    print('line 1')
    print('line 2')

debugger_test()